In [61]:
#import dependencies
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
pd.set_option('display.max_columns', 200)
import re
import collections

In [58]:
#read in data from ?fao.csv? and display head for initial examination
csv_file = "../Resources/fao.csv"
fao_data_df = pd.read_csv(csv_file, encoding="ANSI")
fao_data_df.head()

,Area Abbreviation,Area Code,Area,Item Code,Item,Element Code,Element,Unit,latitude,longitude,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,Y1970,Y1971,Y1972,Y1973,Y1974,Y1975,Y1976,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,Y1986,Y1987,Y1988,Y1989,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
0,AFG,2,Afghanistan,2511,Wheat and products,5142,Food,1000 tonnes,33.94,67.71,1928.0,1904.0,1666.0,1950.0,2001.0,1808.0,2053.0,2045.0,2154.0,1819.0,1963.0,2215.0,2310.0,2335.0,2434.0,2512.0,2282.0,2454.0,2443.0,2129.0,2133.0,2068.0,1994.0,1851.0,1791.0,1683.0,2194.0,1801.0,1754.0,1640.0,1539.0,1582.0,1840.0,1855.0,1853.0,2177.0,2343.0,2407.0,2463.0,2600.0,2668.0,2776.0,3095.0,3249.0,3486.0,3704.0,4164.0,4252.0,4538.0,4605.0,4711.0,4810,4895
1,AFG,2,Afghanistan,2805,Rice (Milled Equivalent),5142,Food,1000 tonnes,33.94,67.71,183.0,183.0,182.0,220.0,220.0,195.0,231.0,235.0,238.0,213.0,205.0,233.0,246.0,246.0,255.0,263.0,235.0,254.0,270.0,259.0,248.0,217.0,217.0,197.0,186.0,200.0,193.0,202.0,191.0,199.0,197.0,249.0,218.0,260.0,319.0,254.0,326.0,347.0,270.0,372.0,411.0,448.0,460.0,419.0,445.0,546.0,455.0,490.0,415.0,442.0,476.0,425,422
2,AFG,2,Afghanistan,2513,Barley and products,5521,Feed,1000 tonnes,33.94,67.71,76.0,76.0,76.0,76.0,76.0,75.0,71.0,72.0,73.0,74.0,71.0,70.0,72.0,76.0,77.0,80.0,60.0,65.0,64.0,64.0,60.0,55.0,53.0,51.0,48.0,46.0,46.0,47.0,46.0,43.0,43.0,40.0,50.0,46.0,41.0,44.0,50.0,48.0,43.0,26.0,29.0,70.0,48.0,58.0,236.0,262.0,263.0,230.0,379.0,315.0,203.0,367,360
3,AFG,2,Afghanistan,2513,Barley and products,5142,Food,1000 tonnes,33.94,67.71,237.0,237.0,237.0,238.0,238.0,237.0,225.0,227.0,230.0,234.0,223.0,219.0,225.0,240.0,244.0,255.0,185.0,203.0,198.0,202.0,189.0,174.0,167.0,160.0,151.0,145.0,145.0,148.0,145.0,135.0,132.0,120.0,155.0,143.0,125.0,138.0,159.0,154.0,141.0,84.0,83.0,122.0,144.0,185.0,43.0,44.0,48.0,62.0,55.0,60.0,72.0,78,89
4,AFG,2,Afghanistan,2514,Maize and products,5521,Feed,1000 tonnes,33.94,67.71,210.0,210.0,214.0,216.0,216.0,216.0,235.0,232.0,236.0,200.0,201.0,216.0,228.0,231.0,234.0,240.0,228.0,234.0,228.0,226.0,210.0,199.0,192.0,182.0,173.0,170.0,154.0,148.0,137.0,144.0,126.0,90.0,141.0,150.0,159.0,108.0,90.0,99.0,72.0,35.0,48.0,89.0,63.0,120.0,208.0,233.0,249.0,247.0,195.0,178.0,191.0,200,200


In [ ]:
# group FAO data by 'Area' (country) and 'Item' (category) 

cntry_item_grp = fao_data_df.groupby(['Area', 'Item']).sum()
cntry_item_grp.head

In [62]:
#create a list from a unique set of the 'Item' categories from initial dataset
# create a dataframe to be exported to csv
FAO_ctgry_df = pd.DataFrame({'FAO_Category': list(set(fao_data_df['Item']))})
#FAO_ctgry_df.to_csv('./../FAO_ctgry_df.csv')
FAO_ctgry_df.head()

,FAO_Categories
0,"Citrus, Other"
1,Cephalopods
2,Groundnut Oil
3,Sugar non-centrifugal
4,Nuts and products


Analyze categories and create column of normalized category 'bins' in excel

In [76]:
#read category datagrame with new column set index to normalized category
csv_file = "./../FAO_norm_ctgry_df.csv"
FAO_nrm_ctgry_df = pd.read_csv(csv_file)
FAO_nrm_ctgry_df = FAO_nrm_ctgry_df.set_index('normalized_category')
FAO_nrm_ctgry_df.head()
    

,FAO_Category
normalized_category,
Oil,"Oilcrops, Other"
Grain,"Pulses, Other and products"
Spice,Pepper
Grain,Cereals - Excluding Beer
Plant,Cassava and products


In [77]:
#print statements to test output of itterows functions on naked row element (nothing in [])
#create list of tuples as key, value pairs to map a dictionary of categories
cat_tuples= []
for index, row in FAO_nrm_ctgry_df.iterrows():
    #print(index)
    #print("--------------------------------")
    #print(row['FAO_Category'])
    #print("--------------------------------")
    cat_tuples.append(tuple([index, row['FAO_Category']]))
cat_tuples

[('Oil', 'Oilcrops, Other'),
 ('Grain', 'Pulses, Other and products'),
 ('Spice', 'Pepper'),
 ('Grain', 'Cereals - Excluding Beer'),
 ('Plant', 'Cassava and products'),
 ('Fruit', 'Dates'),
 ('Seafood', 'Cephalopods'),
 ('Oil', 'Maize Germ Oil'),
 ('Seed', 'Nuts and products'),
 ('Alcohol', 'Cereals, Other'),
 ('Fruit', 'Bananas'),
 ('Dry Goods', 'Cocoa Beans and products'),
 ('Meat', 'Poultry Meat'),
 ('Meat', 'Mutton & Goat Meat'),
 ('Meat', 'Pigmeat'),
 ('Seed', 'Cottonseed'),
 ('Meat', 'Bovine Meat'),
 ('Dry Goods', 'Stimulants'),
 ('Meat', 'Offals'),
 ('Fruit', 'Pineapples and products'),
 ('Oil', 'Fish, Body Oil'),
 ('Dairy', 'Cream'),
 ('Grain', 'Rice (Milled Equivalent)'),
 ('Seafood', 'Aquatic Products, Other'),
 ('Seafood', 'Crustaceans'),
 ('Meat', 'Offals, Edible'),
 ('Oil', 'Cottonseed Oil'),
 ('Seed', 'Coconuts - Incl Copra'),
 ('Oil', 'Ricebran Oil'),
 ('Meat', 'Freshwater Fish'),
 ('Dairy', 'Butter, Ghee'),
 ('Meat', 'Meat, Other'),
 ('Fruit', 'Grapes and products (excl

In [78]:
#create dictionary mapping FAO_Categries to their normalized counterparts
cat_dict = collections.defaultdict(list)
for norm, cat in cat_tuples:
    cat_dict[norm].append(cat)
cat_dict

defaultdict(list,
            {'Oil': ['Oilcrops, Other',
              'Maize Germ Oil',
              'Fish, Body Oil',
              'Cottonseed Oil',
              'Ricebran Oil',
              'Oilcrops',
              'Sunflowerseed Oil',
              'Fish, Liver Oil',
              'Rape and Mustard Oil',
              'Oilcrops Oil, Other',
              'Coconut Oil',
              'Olive Oil',
              'Sesameseed Oil',
              'Palmkernel Oil',
              'Vegetable Oils',
              'Palm Oil',
              'Soyabean Oil',
              'Groundnut Oil'],
             'Grain': ['Pulses, Other and products',
              'Cereals - Excluding Beer',
              'Rice (Milled Equivalent)',
              'Barley and products',
              'Oats',
              'Maize and products',
              'Beans',
              'Rye and products',
              'Pulses',
              'Millet and products',
              'Sorghum and products',
              'Whea

In [79]:
#import second databaes

csv_file = "../Resources/en.openfoodfacts.org.products.tsv"
food_data_df = pd.read_csv(csv_file, sep='\t', low_memory=False)
food_data_df.head()


,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging,packaging_tags,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,no_nutriments,additives_n,additives,additives_tags,additives_en,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_tags,nutrition_grade_uk,nutrition_grade_fr,pnns_groups_1,pnns_groups_2,states,states_tags,states_en,main_category,main_category_en,image_url,image_small_url,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,-butyric-acid_100g,-caproic-acid_100g,-caprylic-acid_100g,-capric-acid_100g,-lauric-acid_100g,-myristic-acid_100g,-palmitic-acid_100g,-stearic-acid_100g,-arachidic-acid_100g,-behenic-acid_100g,-lignoceric-acid_100g,-cerotic-acid_100g,-montanic-acid_100g,-melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,-alpha-linolenic-acid_100g,-eicosapentaenoic-acid_100g,-docosahexaenoic-acid_100g,omega-6-fat_100g,-linoleic-acid_100g,-arachidonic-acid_100g,-gamma-linolenic-acid_100g,-dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,-oleic-acid_100g,-elaidic-acid_100g,-gondoic-acid_100g,-mead-acid_100g,-erucic-acid_100g,-nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,-sucrose_100g,-glucose_100g,-fructose_100g,-lactose_100g,-maltose_100g,-maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
0,0000000003087,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,1474103866,2016-09-17T09:17:46Z,1474103893,2016-09-17T09:18:13Z,Farine de blé noir,NaN,1kg,NaN,NaN,Ferme t'y R'nao,ferme-t-y-r-nao,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:FR,en:france,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"en:to-be-completed, en:nutrition-facts-to-be-c...","en:to-be-completed,en:nutrition-facts-to-be-co...","To be completed,Nutrition facts to be complete...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000004530,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,1489069957,2017-03-09T14:32:37Z,1489069957,2017-03-09T14:32:37Z,Banana Chips Sweetened (Whole),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,en:united-states,United States,"Bananas, vegetable oil (coconut oil, cor

In [85]:
#create category dataframe and append a boolean column stating if category is in english

OFD_ctgry_df = pd.DataFrame({'OFD_Categories': list(set(food_data_df['main_category']))})
#OFD_ctgry_df['en_boolean'] = OFD_ctgry_df['OFD_Categories'].apply(lambda x: 'True' if re.search(r'en:', x) == True else "False")
OFD_ctgry_df['en_boolean'] = OFD_ctgry_df['OFD_Categories'].str.match(r'en:')
OFD_ctgry_df.head()

,OFD_Categories,en_boolean
0,NaN,NaN
1,en:soymilk,True
2,fr:ble-assaisonne,False
3,en:glace-cherries,True
4,fr:salade-de-fruits,False


In [84]:
# reduce dataframe to only english (supposedly, lol) categories and strip the prefix

OFD_en_ctgry_df = OFD_ctgry_df.loc[OFD_ctgry_df['en_boolean'] == True, :].reset_index().drop(columns=['index', 'en_boolean'])
OFD_en_ctgry_df['OFD_Categories'] = OFD_en_ctgry_df['OFD_Categories'].str.slice(3)
OFD_en_ctgry_df.head()

,OFD_Categories
0,soymilk
1,glace-cherries
2,baking
3,productos-lacteos
4,drinking-chocolate


In [ ]:
#define function to comb through the other category list
def food(x):
    oil = [oil,]
    grain = [grain,]
    spice = [spice,]
    plant = [plant, ]
    fruit = [fruit,]
    seafood = []
    soy = []
    spices = []
    vegetable = []
    other = []
    sugar = []
    infant = []
    alcohol = []
    meat = []
    seed = []
    plant = []
    dairy = []
    palm = []
    dry_goods = []
    grain = []
    if

In [86]:
# remove the columns with multirow text as initial cleanup to the massive open food database
mini_food_df = food_data_df[food_data_df.columns.difference(['ingredients_text', 'states', 'states_tags', 'states_en','additives', 'additives_en', 'additives_n' ])]


In [87]:
# define function to determine percentage of missing values in each column
def missing_values_table(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns

In [89]:
#  run reduced dataframe through function to ascertain which columns contain lots of empty values
# sort values in ascending order so cleanest columns are at the top
miss_df = missing_values_table(mini_food_df)
miss_df = miss_df.sort_values('% of Total Values', ascending=True)
miss_df.head() 

Your selected dataframe has 156 columns.
There are 154 columns that have missing values.


,Missing Values,% of Total Values
created_t,3,0.0
url,26,0.0
creator,3,0.0
created_datetime,10,0.0
code,26,0.0


In [91]:
# create new dataframe with only desired columns in desired order with the good old copy-past

food_df = food_data_df[['created_t', 'url', 'creator', 'created_datetime', 'code',
       'countries_en', 'countries_tags', 'countries', 'product_name', 'brands',
       'brands_tags', 'energy_100g', 'proteins_100g', 'salt_100g',
       'sodium_100g', 'ingredients_that_may_be_from_palm_oil_n',
       'ingredients_from_palm_oil_n', 'fat_100g', 'sugars_100g',
       'carbohydrates_100g', 'saturated-fat_100g', 'nutrition-score-fr_100g',
       'nutrition_grade_fr', 'nutrition-score-uk_100g', 'fiber_100g',
       'serving_size', 'additives_tags', 'cholesterol_100g', 'trans-fat_100g',
       'calcium_100g', 'vitamin-c_100g', 'iron_100g', 'vitamin-a_100g',
       'pnns_groups_2', 'pnns_groups_1', 'quantity', 'categories_tags',
       'main_category_en', 'main_category', 'categories', 'categories_en',
       'packaging_tags', 'packaging', 'image_url', 'image_small_url',
       'purchase_places', 'labels_en', 'labels', 'labels_tags', 'generic_name',
       'stores', 'manufacturing_places', 'manufacturing_places_tags',
       'allergens']].copy()
food_df.head()

,created_t,url,creator,created_datetime,code,countries_en,countries_tags,countries,product_name,brands,brands_tags,energy_100g,proteins_100g,salt_100g,sodium_100g,ingredients_that_may_be_from_palm_oil_n,ingredients_from_palm_oil_n,fat_100g,sugars_100g,carbohydrates_100g,saturated-fat_100g,nutrition-score-fr_100g,nutrition_grade_fr,nutrition-score-uk_100g,fiber_100g,serving_size,additives_tags,cholesterol_100g,trans-fat_100g,calcium_100g,vitamin-c_100g,iron_100g,vitamin-a_100g,pnns_groups_2,pnns_groups_1,quantity,categories_tags,main_category_en,main_category,categories,categories_en,packaging_tags,packaging,image_url,image_small_url,purchase_places,labels_en,labels,labels_tags,generic_name,stores,manufacturing_places,manufacturing_places_tags,allergens
0,1474103866,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,2016-09-17T09:17:46Z,0000000003087,France,en:france,en:FR,Farine de blé noir,Ferme t'y R'nao,ferme-t-y-r-nao,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1489069957,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,2017-03-09T14:32:37Z,0000000004530,United States,en:united-states,US,Banana Chips Sweetened (Whole),NaN,NaN,2243.0,3.57,0.00000,0.000,0.0,0.0,28.57,14.29,64.29,28.57,14.0,d,14.0,3.6,28 g (1 ONZ),NaN,0.018,0.0,0.000,0.0214,0.00129,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1489069957,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,2017-03-09T14:32:37Z,0000000004559,United States,en:united-states,US,Peanuts,Torn & Glasser,torn-glasser,1941.0,17.86,0.63500,0.250,0.0,0.0,17.86,17.86,60.71,0.00,0.0,b,0.0,7.1,28 g (0.25 cup),NaN,0.000,0.0,0.071,0.0000,0.00129,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1489055731,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,2017-03-09T10:35:31Z,0000000016087,United States,en:united-states,US,Organic Salted Nut Mix,Grizzlies,grizzlies,2540.0,17.86,1.22428,0.482,0.0,0.0,57.14,3.57,17.86,5.36,12.0,d,12.0,7.1,28 g (0.25 cup),NaN,NaN,NaN,0.143,NaN,0.00514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1489055653,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,2017-03-09T10:34:13Z,0000000016094,United States,en:united-states,US,Organic Polenta,Bob's Red Mill,bob-s-red-mill,1552.0,8.57,NaN,NaN,0.0,0.0,1.43,NaN,77.14,NaN,NaN,NaN,NaN,5.7,35 g (0.25 cup),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
# drop NaN values, group by count of categories_en where count of products > 1, descending 
category_df = food_df.dropna(axis=0, subset=['categories_en'])
grp_ctgry = category_df.groupby(['categories_en'])
grp_cnt = grp_ctgry.count().sort_values('created_t', ascending=False)
grp_cnt_df = pd.DataFrame(grp_cnt['created_t'])
grp_cnt_df = grp_cnt_df.rename(columns={'created_t': 'count'})
grp_cnt_df = grp_cnt_df.loc[grp_cnt_df['count'] > 1, :]
grp_cnt_df

,count
categories_en,
"Beverages,Non-sugared beverages",2345
"Sugary snacks,Biscuits and cakes,Biscuits",893
Fats,732
"Sugary snacks,Chocolates,Dark chocolates",678
"Sugary snacks,Confectioneries,Candies",587
"Groceries,Sauces,Tomato sauces,Ketchup",561
"Salty snacks,Appetizers,Crackers",534
"Dairies,Milks",504
"Sugary snacks,Biscuits and cakes,Biscuits,Chocolate biscuits",498


In [93]:
# drop NaN values, group b brands where count of products > 300, descending 

brand_df = food_df.dropna(axis=0, subset=['brands'])
grp_brnd = brand_df.groupby(['brands'])
brnd_cnt = grp_brnd.count().sort_values('created_t', ascending=False)
brnd_cnt_df = pd.DataFrame(brnd_cnt['created_t'])
brnd_cnt_df = brnd_cnt_df.rename(columns={'created_t': 'count'})
brnd_cnt_df = brnd_cnt_df.loc[brnd_cnt_df['count'] > 300, :]
brnd_cnt_df

,count
brands,
Carrefour,3670
Auchan,3217
U,2608
Casino,2091
Leader Price,2015
Meijer,1995
Kroger,1662
Cora,1374
Ahold,1370


In [94]:
# drop NaN values, group by country where count of products > 300, descending 

country_df = food_df.dropna(axis=0, subset=['countries_en'])
grp_cntry = country_df.groupby(['countries_en'])
cntry_cnt = grp_cntry.count().sort_values('created_t', ascending=False)
cntry_cnt_df = pd.DataFrame(cntry_cnt['created_t'])
cntry_cnt_df = cntry_cnt_df.rename(columns={'created_t': 'count'})
cntry_cnt_df = cntry_cnt_df.loc[cntry_cnt_df['count'] > 1, :]
cntry_cnt_df

,count
countries_en,
United States,173159
France,123961
Switzerland,14932
Germany,8470
Spain,5529
United Kingdom,5092
Belgium,2936
Australia,2201
Russia,1582
